In [111]:

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
import scipy.sparse as sp
import numpy as np

In [112]:
# Relevant parameters to play around with

order = 2 # mesh order
max_h = 0.05 # mesh size
C_w = 1000# penalty term weight

In [113]:
# 1. Define the Geometry
geo = CSGeometry()
box = OrthoBrick(Pnt(-1, -1, -1), Pnt(1, 1, 1))
geo.Add(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))
# Draw(mesh)

In [114]:
V_k = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
V_km1 = H1(mesh, order=order)

prodSpc = V_k * V_km1

omega, sigma = prodSpc.TrialFunction()
eta, tau = prodSpc.TestFunction()

h_min_bnd = min(Integrate(1, mesh, BND, element_wise=True))
# print(h_min_bnd)

In [115]:
a = BilinearForm(prodSpc)

a += sigma * tau * dx
a += - omega * grad(tau) * dx

a +=  grad(sigma) * eta * dx
a +=  curl(omega) * curl(eta) * dx

Lets unpack the first boundary term from (2c):

$$
+ \, \int_{\Gamma} \text{tr}(\star d\omega_h) \wedge \text{tr}(\eta_h) \tag{2c.1}
$$

If we consider the k-forms in 3D to be: 

$$
\omega , \eta \in \Lambda^1(\Omega) \, \Longleftrightarrow \, \omega, \eta \in H(\text{curl}, \Omega)
$$

We can plug in the according vector calculus operations and transform them with help from some identities:

$$
\begin{aligned}
&= \int_{\Gamma} (\text{n} \times \nabla \times \omega_h) \cdot \eta_h \\[10pt]

&= \int_{\Gamma} (\nabla \times \omega_h) \cdot (\eta_h \times \text{n})
\end{aligned}
$$



In [116]:
n = specialcf.normal(mesh.dim)

integrand_2c1 = CF(curl(omega) * Cross(eta, n))
a += integrand_2c1 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))


And the second boundary term from (2c):

$$
\begin{aligned}
&+ \int_{\Gamma} \text{tr} ( \omega_h ) \wedge \text{tr} ( \star d\eta_h ) \tag{2c.2} \\[10pt]

&=\int_{\Gamma} (\omega_h \times \text{n}) \cdot (\nabla \times \eta_h)
\end{aligned}
$$

In [117]:
integrand_2c2 = CF( Cross(omega, n) * curl(eta) )
a += integrand_2c2 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

And the pentalty or stabilization term:

$$
+ \, \frac{C_w}{h} \langle \text{tr}\, \omega, \text{tr}\, \eta \rangle_{L^2(\Gamma)} \tag{2d.1}
$$

In [118]:
h_min_bnd = min(Integrate(1, mesh, BND, element_wise=True))

tr_omega = Cross(eta, n)
tr_eta = Cross(omega, n)

a += (C_w / h_min_bnd) * tr_omega * tr_eta * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

Differential Operator Functions to soften the load on my brain for calculation of hodge laplacian of a manufactured solution

In [119]:
def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u

def GradOfCF(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    grad_u = CF((cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z)))
    return grad_u

def CurlOfCF(cf):
    """ Function to compute the curl of a vector coefficient function using JacobianOfCF """
    Jac_u = JacobianOfCF(cf)
    
    curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                 Jac_u[0,2] - Jac_u[2,0],  
                 Jac_u[1,0] - Jac_u[0,1]))
    
    return curl_u

def DivOfCF(cf):
    """ Function to compute the divergence of a vector coefficient function cf """
    return cf[0].Diff(x) + cf[1].Diff(y) + cf[2].Diff(z)


In [120]:
A = 0.05  # Amplitude of the pulse
sigma_pulse = 0.2  # Width of the Gaussian pulse
r0 = (0.5, 0.5, 0.5)  # Center of the Gaussian pulse
n_pulse = (1, 0, 0)  # Direction of the pulse (unit vector)

gauss_pulse = CF((
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[0],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[1],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[2]
))

# omega_m = CF((sin(pi*x)*sin(pi*y)*sin(pi*z), sin(pi*x)*sin(pi*y)*sin(pi*z), sin(pi*x)*sin(pi*y)*sin(pi*z)))
omega_m = gauss_pulse
hL_omega = CurlOfCF(CurlOfCF(omega_m)) - GradOfCF(DivOfCF(omega_m))


f = LinearForm(prodSpc)
f += hL_omega * eta * dx

In [121]:
a.Assemble()
f.Assemble()

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
#cond_nr = np.linalg.cond(A.todense())

sol = GridFunction(prodSpc)
res = f.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=prodSpc.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res

gf_omega , gf_sigma = sol.components

curl_omega = curl(gf_omega)
grad_sigma = grad(gf_sigma)

print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
#print("Matrix Condition Number: ", cond_nr)
print("Residual: ", Norm(res))
print("L2 Error omega: ", sqrt(Integrate((gf_omega - omega_m)**2, mesh)))
print("L2 Error curl(omega): ", sqrt(Integrate((curl(gf_omega) - CurlOfCF(omega_m))**2, mesh)))
print("L2 Error sigma:: ", sqrt(Integrate((gf_sigma + DivOfCF(omega_m))**2, mesh)))
print("L2 Error grad(sigma): ", sqrt(Integrate((grad(gf_sigma) + GradOfCF(DivOfCF(omega_m)))**2, mesh)))



Matrix dimensions: 47901 x 47901
Residual:  4.271296599396992e-09
L2 Error omega:  0.0007759899579836811
L2 Error curl(omega):  0.004743346490355457
L2 Error sigma::  0.0020054271293129433
L2 Error grad(sigma):  0.05761681258359089
